<a href="https://colab.research.google.com/github/velmer/retrieval-information/blob/master/lab-04/ri_lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import nltk
import time
import heapq
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

_stopwords = set(stopwords.words('portuguese'))

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Carregando Coleção

In [7]:
data = pd.read_csv("https://raw.githubusercontent.com/velmer/retrieval-information/master/lab-01/output/results.csv")
data.head()

,title,sub_title,author,date,section,text,url
0,VÍDEO – “Estarei nas ruas com a militância do ...,Não possui,Pedro Zambarda de Araujo,05/04/2019 22:44:19,Não possui,Não ficaremos calados diante da injustiça. Lul...,https://www.diariodocentrodomundo.com.br/video...
1,Essencial do DCM – Bolsonaro diz que não nasce...,Não possui,Pedro Zambarda de Araujo,05/04/2019 21:00:02,Não possui,O jornalismo do DCM precisa de você para conti...,https://www.diariodocentrodomundo.com.br/essen...
2,Conheça o deputado árabe que pode derrubar o p...,Não possui,Diario do Centro do Mundo,05/04/2019 22:00:56,Não possui,A revista francesa L’Obs desta semana traz um ...,https://www.diariodocentrodomundo.com.br/conhe...
3,"VÍDEO – Guedes, as tchutchucas e a lacrada des...",Não possui,Nathalí Macedo,05/04/2019 16:23:14,Não possui,O jornalismo do DCM precisa de você para conti...,https://www.diariodocentrodomundo.com.br/video...
4,O que está por trás da articulação para Lula t...,Não possui,Joaquim de Carvalho,05/04/2019 21:42:17,Não possui,Veja o vídeo “Foi para isso que prenderam o Lu...,https://www.diariodocentrodomundo.com.br/o-que...


# Questão 1

In [8]:

tokenData = pd.DataFrame(data['text'], columns=['text'])
tokenData['tokenized_text'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)
tokenData.head()

,text,tokenized_text
0,Não ficaremos calados diante da injustiça. Lul...,"[não, ficaremos, calados, diante, da, injustiç..."
1,O jornalismo do DCM precisa de você para conti...,"[o, jornalismo, do, dcm, precisa, de, você, pa..."
2,A revista francesa L’Obs desta semana traz um ...,"[a, revista, francesa, l, ’, obs, desta, seman..."
3,O jornalismo do DCM precisa de você para conti...,"[o, jornalismo, do, dcm, precisa, de, você, pa..."
4,Veja o vídeo “Foi para isso que prenderam o Lu...,"[veja, o, vídeo, “, foi, para, isso, que, pren..."


In [0]:
def is_tokenized_word(word):
  """
    Returns if the specified word matches all requirements
    to be a valid token.
  """
  return len(word) > 1 and any(c.isalnum() for c in word)

In [0]:
def build_index():
  index = {}
  num_document = 0

  for text in tokenData.tokenized_text:
    num_document += 1
    words_set = set(text)
    for word in words_set:
      if is_tokenized_word(word) and word.lower() not in _stopwords:
        if word not in index.keys():
          index[word] = []
        index[word].append((num_document, text.count(word)))

  return index

In [13]:
indexed_data = pd.DataFrame()
index = build_index()
indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents (With Count)'] = [docs for docs in index.values()]
indexed_data.to_csv('indexed_data.csv', encoding='utf-8', index=False)
indexed_data.head(10)

,Word,Documents (With Count)
0,nacional,"[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 5), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 5), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 2), (60, 1), (61, 6), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 3), (100, 1), ...]"
1,militância,"[(1, 1), (25, 1), (108, 1), (309, 1), (407, 4)]"
2,estarei,"[(1, 1), (102, 2)]"
3,preso,"[(1, 1), (52, 1), (55, 2), (57, 1), (62, 3), (108, 5), (119, 1), (146, 1), (167, 3), (198, 3), (217, 1), (219, 3), (224, 1), (227, 2), (229, 1), (236, 3), (276, 2), (278, 1), (300, 1), (327, 1), (367, 1), (368, 2), (372, 1), (404, 1), (406, 2), (416, 1), (442, 1), (457, 1), (480, 1), (513, 1), (519, 3), (525, 6), (577, 1), (611, 1), (627, 2), (632, 1)]"
4,lula,"[(1, 1), (5, 1), (27, 9), (30, 1), (37, 16), (52, 6), (57, 2), (62, 19), (68, 1), (102, 1), (108, 19), (130, 1), (133, 5), (137, 1), (138, 4), (146, 77), (174, 8), (182, 5), (185, 16), (198, 2), (212, 1), (224, 8), (229, 1), (233, 1), (236, 3), (248, 1), (252, 2), (254, 1), (268, 1), (276, 15), (283, 1), (285, 1), (300, 1), (304, 1), (305, 1), (308, 1), (357, 4), (361, 1), (367, 2), (374, 1), (379, 1), (406, 1), (445, 1), (457, 14), (463, 2), (480, 4), (484, 5), (513, 3), (519, 4), (523, 3), (525, 2), (552, 1), (579, 1), (586, 1), (591, 1), (603, 1), (627, 1), (632, 2), (635, 1)]"
5,ponto,"[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 3), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 3), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 1), ...]"
6,ficaremos,"[(1, 1)]"
7,bolsonaro,"[(1, 1), (6, 1), (10, 16), (11, 1), (13, 3), (15, 1), (17, 7), (19, 1), (25, 1), (27, 15), (28, 1), (29, 1), (30, 4), (37, 1), (46, 1), (48, 2), (51, 3), (52, 1), (57, 14), (59, 5), (66, 1), (68, 1), (71, 4), (72, 1), (77, 1), (81, 4), (97, 3), (98, 2), (99, 6), (104, 3), (113, 1), (114, 6), (121, 6), (123, 1), (129, 1), (130, 1), (133, 1), (135, 1), (136, 1), (138, 9), (145, 4), (146, 1), (147, 6), (153, 1), (157, 1), (166, 3), (167, 1), (171, 3), (174, 5), (177, 1), (182, 6), (185, 1), (187, 3), (192, 5), (193, 2), (194, 2), (195, 1), (208, 2), (212, 6), (214, 4), (223, 3), (226, 1), (229, 7), (232, 4), (233, 1), (235, 2), (236, 7), (238, 1), (239, 4), (241, 1), (243, 4), (246, 1), (248, 1), (249, 3), (250, 5), (251, 1), (252, 4), (255, 7), (256, 2), (263, 1), (264, 3), (268, 4), (277, 4), (278, 3), (279, 3), (282, 8), (283, 4), (285, 3), (288, 1), (289, 11), (297, 10), (300, 1), (303, 1), (304, 1), (306, 4), (308, 2), (309, 10), (315, 2), (316, 1), (319, 1), ...]"
8,alterar,"[(1, 1), (210, 1), (229, 1), (243, 1)]"
9,neste,"[(1, 1), (33, 2), (

# Questão 2

In [0]:
def document_at_time(query, index, k):
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_pos in range(1, len(data) + 1):
        sd = 0
        for document_count in document_count_list:
            for i in document_count:
                if (i[0] == document_pos):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, document_pos))
    
    return heapq.nlargest(k, r)
  
def term_at_time(query, index, k):
    a = {}
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_count in document_count_list:
        for post in document_count:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))

    return heapq.nlargest(k, r)

In [0]:
queries = ["presidente", "bolsonaro", "país", "futebol", "site"]
k = 10
results_doc = []
results_term = []

for query in queries:
  score_doc = document_at_time(query, index, k)
  results_doc.append(score_doc)
  score_term = term_at_time(query, index, k)
  results_term.append(score_term)

queries_df = pd.DataFrame()
queries_df['Consulta'] = queries
queries_df['Documento por vez'] = results_doc
queries_df['Termo por vez'] = results_term
queries_df['Mesmo resultado'] = queries_df['Documento por vez'] == queries_df['Documento por vez']
queries_df

,Consulta,Documento por vez,Termo por vez,Mesmo resultado
0,presidente,"[(13, 525), (12, 583), (11, 57), (10, 514), (9, 61), (9, 17), (8, 138), (8, 51), (8, 27), (7, 471)]","[(13, 525), (12, 583), (11, 57), (10, 514), (9, 61), (9, 17), (8, 138), (8, 51), (8, 27), (7, 471)]",True
1,bolsonaro,"[(17, 577), (16, 10), (15, 27), (14, 57), (12, 627), (12, 526), (11, 289), (10, 514), (10, 309), (10, 297)]","[(17, 577), (16, 10), (15, 27), (14, 57), (12, 627), (12, 526), (11, 289), (10, 514), (10, 309), (10, 297)]",True
2,país,"[(13, 525), (9, 627), (8, 519), (8, 278), (7, 583), (6, 309), (6, 243), (6, 179), (5, 167), (5, 138)]","[(13, 525), (9, 627), (8, 519), (8, 278), (7, 583), (6, 309), (6, 243), (6, 179), (5, 167), (5, 138)]",True
3,futebol,"[(7, 127), (3, 165), (2, 171), (2, 160), (1, 477), (1, 174), (1, 146)]","[(7, 127), (3, 165), (2, 171), (2, 160), (1, 477), (1, 174), (1, 146)]",True
4,site,"[(6, 289), (2, 627), (2, 594), (2, 577), (2, 378), (2, 302), (2, 134), (1, 623), (1, 606), (1, 596)]","[(6, 289), (2, 627), (2, 594), (2, 577), (2, 378), (2, 302), (2, 134), (1, 623), (1, 606), (1, 596)]",True


In [0]:
import time
        
def document_at_time_1000_times_per_query():
  for query in queries:
    for _ in range(10000):
      document_at_time(query, index, k)

def term_at_time_1000_times_per_query():
  for query in queries:
    for _ in range(1000):
      term_at_time(query, index, k)
      
qty_queries_execution = 1000 * 5

document_start = time.time()
document_at_time_1000_times_per_query()
document_time_elapsed = time.time() - document_start
document_average_time = document_time_elapsed / qty_queries_execution
print("Tempo médio para execução de Documento Por Vez: ", document_average_time)

term_start = time.time()
term_at_time_1000_times_per_query()
term_time_elapsed = time.time() - term_start
term_average_time = term_time_elapsed / qty_queries_execution
print("Tempo médio para execução de Termo Por Vez: ", term_average_time)

Tempo médio para execução de Documento Por Vez:  0.04084352173805237
Tempo médio para execução de Termo Por Vez:  0.00010008358955383301


Como esperado, o tempo médio para execução de Termo Por Vez foi consideravelmente menor que o de Documento Por Vez, sendo cerca de 400x mais rápido.

# Questão 3

In [0]:
def conjuctive_query(query, indexes, k):
  query_indexes = []
  rank = []
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort()
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

In [0]:
conjuctives_queries = ["brasil justiça", "lula livre", "presidente juiz", "jair bolsonaro", "país política"]

k = 10
conjuctives_results = []

for query in conjuctives_queries:
  conjuctive_score = conjuctive_query(query, index, k)
  conjuctives_results.append(conjuctive_score)

conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = conjuctives_queries
conj_queries_df['Results'] = conjuctives_results
conj_queries_df

,Query,Results
0,brasil justiça,"[(17, 525), (15, 146), (11, 37), (9, 577), (9, 545), (9, 266), (9, 192), (9, 51), (8, 627), (8, 130)]"
1,lula livre,"[(120, 146), (21, 108), (18, 37), (17, 276), (16, 457), (9, 174), (7, 357), (6, 484), (4, 252), (4, 236)]"
2,presidente juiz,"[(9, 212), (8, 282), (6, 229), (6, 219), (3, 480), (3, 424), (3, 37), (2, 584), (2, 520), (2, 179)]"
3,jair bolsonaro,"[(23, 577), (21, 10), (16, 289), (16, 57), (16, 27), (14, 526), (13, 627), (11, 514), (11, 457), (11, 309)]"
4,país política,"[(23, 138), (18, 525), (12, 627), (12, 519), (10, 179), (10, 174), (9, 583), (9, 572), (9, 249), (8, 545)]"
